In [1]:
import numpy as np
import pandas as pd
import oenotebook as oenb
from openeye.oechem import *
from openmoltools.openeye import *
from openeye.oeiupac import *

### Part 1: Cleaning Table from Paper

In [2]:
#Upload all three tables from Kojima et al, 1997 paper
table_1 = pd.read_excel('tables/table1_cleaned.xlsx')
table_2 = pd.read_excel('tables/table2_cleaned.xlsx')
table_3 = pd.read_excel('tables/table3_cleaned.xlsx')

In [3]:
#Reformat tables to have the same column headings so they can be concatentated
table_2 = table_2.rename(columns={'Solvent': 'Solute'})
table_2 = table_2.rename(columns={'Ref': 'Ref.'})
table_2['Method'] = ['NaN' for x in table_2.Solute]
table_2['Uncertain'] = ['0.0' for x in table_2.Solute]
table_3 = table_3.rename(columns={'Solvent': 'Solute'})
cols = table_2.columns.tolist()
cols = cols[:3] + [cols[5]] + [cols[3]] + [cols[4]] + [cols[6]]
table_2 = table_2[cols]

In [4]:
#Create the fully concatenated data table
merged = [table_1, table_2, table_3]
table = pd.concat(merged)

#Reset index values 
table = table.reset_index(drop=True)

In [5]:
#Fix formatting of certain compound names
def fix_names(name):
    if name[-1] == " ":
        name = name.rstrip() #removes any spaces at the end of string
    return name

table.Solute = table.Solute.apply(fix_names)

In [6]:
table.head()

,Solute,Temp,Measured,Method,Ref.,Notes,Uncertain
0,Pentane,293.15,945.0,GC,47,NaN,0
1,Pentane,303.15,796.0,GC,47,NaN,0
2,Pentane,313.15,517.0,GC,47,NaN,0
3,Hexane,293.15,2940.0,GC,47,NaN,0
4,Hexane,298.15,40000.0,--,10,NaN,0


In [6]:
#Find solute names that cannot be converted into SMILES
smiles = {}
for name in table.Solute.unique():
    mol_from_name = OEMol()
    OEParseIUPACName(mol_from_name, name)
    smiles[name] = OECreateCanSmiString(mol_from_name)

In [7]:
#Collects all indices for compounds with the following names 
##Maybe find a way to collect the above error names without hardcoding
trouble_names = ['Propionic Acid, 2-Methyl, -Methyl', '1,8-Nonanediyne', '1,2-Benzanthracene', '3,4-Benzopyrene',
            'Benzo[ghi]perylene', 'Sec-butyl methyl, 2-methyl ether', 'PEG 400', 'PEG 600', 'PEG 1000',
            'PEG 1500', 'PEG 7500', 'Tetraethylene Glycol-Dimethyl Ether', 'Tripropylene Glycol-Dimethyl Ether',
            'Triethylene Glycol-Dibutyl Ether', 'Diethylene Glycol-Dibutyl Ether', 'Tripropylene Glycol-Dibutyl Ether',
            'Phosphoric Acid, Tri-(2-Butoxyethyl) Ester']
total = []
for name in trouble_names:
    indices = list(table[table.Solute == name].index)
    total.append(indices)
final_indices = [val for sublist in total for val in sublist]

In [8]:
print('Original Table:', len(table))

#Create a new table without any of the compounds with no discerable IUPAC name
table2 = table.drop(table.index[final_indices])
print('New Table:', len(table2))

Original Table: 1464
New Table: 1374


In [9]:
table2.head()

,Solute,Temp,Measured,Method,Ref.,Notes,Uncertain
0,Pentane,293.15,945.0,GC,47,NaN,0
1,Pentane,303.15,796.0,GC,47,NaN,0
2,Pentane,313.15,517.0,GC,47,NaN,0
3,Hexane,293.15,2940.0,GC,47,NaN,0
4,Hexane,298.15,40000.0,--,10,NaN,0


In [10]:
table2['SMILES'] = [smiles[name] for name in table2.Solute]
cols = table2.columns.tolist()
cols = [cols[0]] + [cols[-1]] + cols[1:3] + [cols[-2]] + cols[3:-2]
table2 = table2[cols]

In [11]:
table2.head()
print(len(table2.Solute.unique()))

435


In [12]:
#Change all solute names to be lower case for consistency
table2.Solute = [name.lower() for name in table2.Solute]
table2.head()
print(len(table2.Solute.unique()))

427


In [13]:
#Find compounds with different names but same SMILES sequence
bad = []
for smile in table2.SMILES.unique():
    group = table2[table2.SMILES == smile]
    if len(group['Solute'].unique()) > 1:
        bad.append(smile)
print(len(bad))

11


In [14]:
#Change compound names to be consistent
table2.at[65:67, 'Solute'] = '1,3,5-trimethylbenzene'
table2.at[1187:1189, 'Solute'] = 'bromoform'
table2.at[249:251, 'Solute'] = 'bromoethane'
table2.at[278:279, 'Solute'] = '1-bromobutane'
table2.at[1047, 'Solute'] = '1,2-dichlorobenzene'
table2.at[463:467, 'Solute'] = '2-butoxyethanol'
table2.at[659, 'Solute'] = '3-methylbutyl acetate'
table2.at[670:673, 'Solute'] = 'dipropyl ether'
table2.at[1160, 'Solute'] = 'dimethyl sulfide'
table2.at[1161, 'Solute'] = 'diethyl sulfide'

In [15]:
table2.to_pickle("./table_cleaned.pkl")

Save `table2` as the final raw output table